In [1]:
!pip install ultralytics gradio -q

In [2]:
pip install fastapi uvicorn gradio python-multipart


Note: you may need to restart the kernel to use updated packages.


In [1]:
from fastapi import FastAPI
import gradio as gr
from ultralytics import YOLO
import cv2
import os
import zipfile
import json
import csv
import tempfile
import shutil
import time
import threading
import datetime

# ----------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------
MODEL_PATH = "C:\\Users\\HP\\Downloads\\best.pt"

# Hardcoded detection settings
DEFAULT_CONF = 0.25
DEFAULT_IOU = 0.45

# File cleanup settings (30 minutes)
FILE_LIFETIME_SECONDS = 1800 
CLEANUP_INTERVAL_SECONDS = 600  

# Colors for bounding boxes (B, G, R)
COLOR_MAP = {
    "short": (0, 0, 255),          # Red (Critical)
    "open_circuit": (255, 0, 0),   # Blue
    "mouse_bite": (19, 69, 139),   #Brown
    
    "spur": (255, 0, 255),         # Magenta
    "missing_hole": (0, 165, 255), # Orange
    "default": (0, 255, 0)         # Green
}

# ----------------------------------------------------------
# BACKGROUND CLEANUP SYSTEM
# ----------------------------------------------------------
tracked_temp_folders = []

def cleanup_loop():
    """Background thread to auto-delete old temp folders."""
    while True:
        try:
            now = time.time()
            for folder_data in tracked_temp_folders[:]:
                path, creation_time = folder_data
                if now - creation_time > FILE_LIFETIME_SECONDS:
                    if os.path.exists(path):
                        shutil.rmtree(path, ignore_errors=True)
                    tracked_temp_folders.remove(folder_data)
        except Exception:
            pass
        time.sleep(CLEANUP_INTERVAL_SECONDS)

cleanup_thread = threading.Thread(target=cleanup_loop, daemon=True)
cleanup_thread.start()

# ----------------------------------------------------------
# LOAD MODEL
# ----------------------------------------------------------
model = None
if os.path.exists(MODEL_PATH):
    try:
        model = YOLO(MODEL_PATH)
        print(f"✅ Model loaded successfully: {MODEL_PATH}")
    except Exception as e:
        print(f"❌ Error loading YOLO model: {e}")
else:
    print(f"⚠️ Warning: Model file '{MODEL_PATH}' not found.")

# ----------------------------------------------------------
# HELPER FUNCTIONS
# ----------------------------------------------------------
def get_color(class_name):
    key = class_name.lower().replace(" ", "_")
    return COLOR_MAP.get(key, COLOR_MAP["default"])

def detect_image(image_path, output_folder, log_file_path):
    """Detects defects, appends table to log file, and saves image."""
    if model is None:
        return None, [], []

    img = cv2.imread(image_path)
    if img is None:
        return None, [], []

    # Run YOLO inference
    results = model(img, conf=DEFAULT_CONF, iou=DEFAULT_IOU)[0]
    
    detections_json = []
    detections_csv = []
    filename = os.path.basename(image_path)

    # ---------------------------------------------------------
    # 1. BUILD TABLE STRING
    # ---------------------------------------------------------
    table_output = []
    table_output.append(f"\n{'='*70}")
    table_output.append(f"📄 RESULTS FOR: {filename}")
    table_output.append(f"{'-'*70}")
    table_output.append(f"{'CLASS NAME':<20} | {'CONFIDENCE':<12} | {'BBOX (x1, y1, x2, y2)':<25}")
    table_output.append(f"{'-'*70}")

    for box in results.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        xyxy = box.xyxy[0].cpu().numpy().astype(int)
        class_name = model.names[cls_id]

        # Add Row to Table
        bbox_str = f"[{xyxy[0]}, {xyxy[1]}, {xyxy[2]}, {xyxy[3]}]"
        table_output.append(f"{class_name:<20} | {conf:<12.4f} | {bbox_str:<25}")

        # Collect Data
        detections_json.append({
            "class": class_name,
            "confidence": round(conf, 4),
            "bbox": xyxy.tolist()
        })
        detections_csv.append([filename, class_name, round(conf, 4), *xyxy])

        # ---------------------------------------------------------
        # 2. DRAW VISUALS (No Background, White Text)
        # ---------------------------------------------------------
        x1, y1, x2, y2 = xyxy
        color = get_color(class_name)
        
        # Dynamic sizing
        h, w, _ = img.shape
        thick = max(int((w + h) / 1000), 2)
        font_scale = max((w + h) / 1000, 1.0) 

       # 1. Draw the bounding box
        cv2.rectangle(img, (x1, y1), (x2, y2), color, thick)
        
        # 2. Define the label text
        label_text = f"{class_name} {conf:.2f}"
        
        # 3. Calculate text size so we know how big the background box needs to be
        (w_text, h_text), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thick)
        
        # 4. Draw the FILLED rectangle (The Background Box)
        #    Coordinates: Top-left of text to Bottom-right of text
        cv2.rectangle(img, (x1, y1 - h_text - 10), (x1 + w_text, y1), color, -1)
        
        # 5. Draw the Text in WHITE (255, 255, 255) on top of the box
        cv2.putText(img, label_text, (x1, y1 - 5), 
                    cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), thick+3)

    table_output.append(f"{'='*70}\n")
    
    # ---------------------------------------------------------
    # 3. WRITE TO LOG FILE (Inside Temp Folder)
    # ---------------------------------------------------------
    final_log_string = "\n".join(table_output)
    print(final_log_string) # Still print to console
    
    with open(log_file_path, "a", encoding="utf-8") as log_f:
        log_f.write(final_log_string + "\n")

    output_img_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_img_path, img)

    return output_img_path, detections_json, detections_csv

# ----------------------------------------------------------
# MAIN PROCESS LOGIC
# ----------------------------------------------------------
def process_images(image_paths, progress=None):
    if not image_paths:
        return [], None, "Please upload images."
    if model is None:
        return [], None, "Model not loaded."

    temp_dir = tempfile.mkdtemp()
    tracked_temp_folders.append((temp_dir, time.time()))

    try:
        detected_dir = os.path.join(temp_dir, "detected")
        reports_dir = os.path.join(temp_dir, "reports")
        os.makedirs(detected_dir, exist_ok=True)
        os.makedirs(reports_dir, exist_ok=True)

        # Create the Text Log File inside the temp directory
        session_log_path = os.path.join(temp_dir, "scan_results.txt")
        with open(session_log_path, "w", encoding="utf-8") as f:
            f.write(f"PCB DEFECT SCAN REPORT\nGenerated: {datetime.datetime.now()}\n\n")

        output_images = []
        all_csv_data = [["Filename", "Class", "Confidence", "x1", "y1", "x2", "y2"]]
        defect_counts = {}

        # Loop with Progress Bar
        for path in image_paths:
            # Pass the session_log_path to the detection function
            out_path, json_data, csv_data = detect_image(path, detected_dir, session_log_path)
            
            if out_path:
                output_images.append(out_path)
                all_csv_data.extend(csv_data)
                
                for item in json_data:
                    c_name = item['class']
                    defect_counts[c_name] = defect_counts.get(c_name, 0) + 1

                json_name = os.path.splitext(os.path.basename(path))[0] + ".json"
                with open(os.path.join(reports_dir, json_name), "w") as f:
                    json.dump(json_data, f, indent=4)

        # Save Master CSV
        csv_path = os.path.join(reports_dir, "summary_report.csv")
        with open(csv_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerows(all_csv_data)

        # Create ZIP Bundle
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        zip_filename = f"PCB_Report_{timestamp}.zip"
        zip_path = os.path.join(temp_dir, zip_filename)
        
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            # 1. Add the text log file
            zipf.write(session_log_path, arcname="scan_results.txt")

            # 2. Add detected images
            for root, _, files in os.walk(detected_dir):
                for file in files:
                    zipf.write(os.path.join(root, file), arcname=f"detected_images/{file}")
            
            # 3. Add reports (JSON/CSV)
            for root, _, files in os.walk(reports_dir):
                for file in files:
                    zipf.write(os.path.join(root, file), arcname=f"reports/{file}")

        total = sum(defect_counts.values())
        summary = f"✅ Processed {len(output_images)} images.\n🔍 Found {total} defects.\n\n"
        for k, v in defect_counts.items():
            summary += f"• {k}: {v}\n"

        return output_images, zip_path, summary

    except Exception as e:
        return [], None, f"Error: {str(e)}"

# ----------------------------------------------------------
# GRADIO UI
# ----------------------------------------------------------
with gr.Blocks() as demo:

    gr.Markdown(
        """
        #  **PCB Defect Detection**
        ###
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            input_files = gr.Files(label="Upload Images", file_types=["image"])
            run_btn = gr.Button("▶️ SUBMIT", variant="primary")
            clear_btn = gr.Button("♻️ RESET", variant="secondary")

        with gr.Column(scale=2):
            summary_box = gr.Textbox(label="📊 ANALYSIS SUMMARY", lines=6)
            output_gallery = gr.Gallery(label="VISUAL RESULTS", columns=3, height="auto")
            zip_download = gr.File(label="📥 Download Report (ZIP)")

    run_btn.click(
        fn=process_images,
        inputs=[input_files],
        outputs=[output_gallery, zip_download, summary_box]
    )

    clear_btn.click(
        lambda: (None, None, None, ""),
        inputs=[],
        outputs=[input_files, output_gallery, zip_download, summary_box]
    )

app = FastAPI(title="PCB Defect Detection Backend")

app = gr.mount_gradio_app(
    app,
    demo,
    path="/"
)


✅ Model loaded successfully: C:\Users\HP\Downloads\best.pt
new /


In [ ]:
!uvicorn pcb:app --reload
